In [1]:
import pandas as pd
import ast
import asyncio
import os
from typing import List, Dict
from dotenv import load_dotenv
from openai import AsyncOpenAI
from tqdm.asyncio import tqdm_asyncio
import json

In [2]:
# Загрузка данных из xlsx файла
df = pd.read_excel('webglm_full.xlsx')

In [3]:
def create_llama_dataset(df: pd.DataFrame) -> List[Dict]:
    dataset = []
    for _, row in df.iterrows():
        question = row['question']
        answer = str(row['answer'])
        examples = ast.literal_eval(row["references"]) if isinstance(row['references'], str) else []
        
        dataset_item = {
            "question": question,
            "answer": answer,
            "references": examples
        }
        dataset.append(dataset_item)
    return dataset


dataset = create_llama_dataset(df)
len(dataset)

43579

In [4]:
load_dotenv()
API_KEY = os.getenv("NEBIUS_API_KEY")

client = AsyncOpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=API_KEY
)

COT_SYSTEM_PROMPT = (
    "Ты умный ассистент, который дополняет ответ последовательностью рассуждений.\n"
)

In [ ]:
def get_cot_prompt(context_list, question, answer):
    context = ''
    for i, c in enumerate(context_list):
        context += f'Источник [{i+1}]:'+"\n"+c+"\n\n"

    prompt = (
        f"# Контекстная информация:\n\n{context}\n\n"
        f"# Вопрос:\n\n{question}\n"
        f"# Оригинальный ответ:\n\n{answer}\n"
        f"# Задача:\n\n"
        "Твоя задача состоит в том, чтобы дать более детальный ответ" \
        "на поставленный вопрос, используя исключительно информацию из контекста.\n" \
        "Для этого последовательно порассуждай про то, какую информацию хочет узнать пользователь" \
        "и как информация из оригинального ответа с этим соотносится.\n" \
        "В своем ответе не ссылайся на оригинальный ответ, но учти, что он абсолютно корректен " \
        "и тебе нужно лишь дать более расширенную его версию.\n"
        "Указывай номер источника, про который рассуждаешь.\n" 
        "Последний абзац должен начинаться со слова ОТВЕТ:\\n и содержать полный ответ на поставленный вопрос.\n"
        "В нем особенно важно указывать номера использованных источников." \
        f"# Твой расширенный ответ:\n\n"
    )
    return prompt

In [6]:
async def run_llm(user_prompt : str, model : str, semaphore : asyncio.Semaphore, system_prompt : str = None):
    async with semaphore:
        messages = []
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        
        messages.append({"role": "user", "content": user_prompt})

        return await client.chat.completions.create(
            model=model,
            messages=messages, 
            temperature=0.6,
            max_completion_tokens=1200
        )
    

async def process_task(task_id, user_prompt, model, semaphore, system_prompt=None):
    try:
        response = await run_llm(
            user_prompt=user_prompt,
            model=model,
            semaphore=semaphore,
            system_prompt=system_prompt
        )
        return {
            "task_id": task_id,
            "content": response.choices[0].message.content,
            "status": "success"
        }
    except Exception as e:
        return {
            "task_id": task_id,
            "error": str(e),
            "status": "failed"
        }

In [7]:
semaphore = asyncio.Semaphore(600)
offset = 40000
package_size = 5000

tasks = [process_task(
    task_id=offset + i,
    user_prompt=get_cot_prompt(row["references"], row["question"], row["answer"]),
    model="meta-llama/Llama-3.3-70B-Instruct",
    semaphore=semaphore,
    system_prompt=COT_SYSTEM_PROMPT)
    for i, row in enumerate(dataset[offset : min(offset+package_size, len(dataset))])
]

results = []
with tqdm_asyncio(total=len(tasks), desc="Processing LLM requests") as pbar:
    for coro in asyncio.as_completed(tasks):
        result = await coro
        results.append(result)
        pbar.update(1)

Processing LLM requests: 100%|██████████| 3579/3579 [10:06<00:00,  5.90it/s]


In [8]:
results.sort(key=lambda x: x['task_id'])

for result in results:
    if result['status'] == "failed":
        print(result['task_id'])

In [9]:
with open("webglm_full_cot.jsonl", "a") as json_file:
    for result in results:
        data = ({
            "question": dataset[result['task_id']]['question'],
            "answer": result['content'],
            "references": dataset[result['task_id']]['references']
        })

        json.dump(data, json_file)
        json_file.write('\n')

In [10]:
with open("webglm_full_cot.jsonl", "r") as file:
    row = json.loads(file.readlines()[39999])

print(row['question'])
print('\n\n')
print(row['answer'])
print('\n\n')
print(row['references'])

Я читаю очень глупую книгу, но в ней есть часть, относящаяся к статистике...



Пользователь упоминает, что он читает книгу, относящуюся к статистике, и хочет узнать больше о ней. Из контекста мы понимаем, что книга, вероятно, не о технической стороне статистики, а скорее о том, как критически мыслить, когда сталкиваешься с данными и статистикой [2]. Это означает, что книга может быть более ориентирована на понимание того, как статистика и ее представление могут быть использованы для обмана или для предоставления ценной информации [1].

Книга, вероятно, написана экономистом, который также является обозревателем и вещателем, и, возможно, почетным членом Королевского статистического общества [3]. Она состоит из 10 правил, которые помогают читателю понять, как критически мыслить, когда он видит статистику, и как проверить ее правдивость [3]. Эти правила не требуют технических знаний, таких как формулы или понимание p-значения или среднего против медианы [2], [3].

Книга также может содерж

In [12]:
with open("webglm_full_cot.jsonl", "r") as file:
    print(len(file.readlines()))

43579
